In [4]:
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt
import numba

# Education Subsidies with Private Information

# Insuring Preference Shocks

# Computing Private Information Problems

In [37]:
def makegrid(N):
    a = np.array([0.2, 0.4])

    q = np.array([1, 2])

    c = np.linspace(.1, 2, N)
    w = np.linspace(0, 1.1, N)
    
    return a, q, c

a, q, c = makegrid(20)

Π = np.array([[0.8, 0.2],
            [0.2, 0.9]])   # First row for a = 0.2, first column for q = 1

@numba.njit()
def utility(c,a):
    return np.sqrt(c) * (1 - a)



## Write down constraints

In [56]:
w0 = np.array([1])

In [57]:
#1 Maximization q -c
factor = np.kron(q,np.ones(len(c))) - np.kron(np.ones(len(q)),c)
factor = np.kron(factor, np.ones(len(a)))

#2 Probability distribution constraint
A_ub = np.ones(len(factor))

# Bayes Rule constraint
# sum over 

#3 Need a matrix here
pb = np.ravel(Π)   # Loop over a first, then q


#4 promise keeping constraint
# Vector of dimension


b = np.concatenate((np.array([1]), np.zeros(len(q)*len(a)), - w0, np.zeros(len(a)*(len(a)-1))))

In [60]:
print(b)

[ 1.  0.  0.  0.  0. -1.  0.  0.]


In [ ]:
res = linprog(factor, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds])

In [46]:
np.ravel(Π)

array([0.8, 0.2, 0.2, 0.9])